In [1]:
import os
import sys
sys.path.insert(0, "..")

import inspect
import torch
import wandb

from notebook_utils import *

from safetensors import safe_open

from models import *
from my_datasets import *
from experiments import *

In [2]:
torch.manual_seed(1234)
ns_model, dataset = quickload_next_state_model_and_dataset()

wandb: Downloading large artifact model-SynNS_gpt2_d1024_L2_H4__nv128_nr16-64_ns8-32_ap0.30-0.50_bp0.20-0.30_sp0.50-0.50_ntr32768_ntt8192:v0, 297.94MB. 2 files... 
wandb:   2 of 2 files downloaded.  
Done. 0:0:0.5


In [3]:
atk_model = ForceOutputAttackerModel(
    input_dim = ns_model.input_dim,
    output_dim = ns_model.num_labels,
    num_outputs = 3)

In [4]:
batch_size = 16
vtgt = torch.randint(0,2, (batch_size, dataset.num_vars))
u = torch.stack([dataset[i]["tokens"] for i in range(batch_size)])

In [5]:
vtgt.shape, u.shape

(torch.Size([16, 128]), torch.Size([16, 96, 257]))

In [6]:
atk_out = atk_model(u, vtgt)

x: torch.Size([16, 97, 768])
